In [2]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("ExtractArtistNames") \
    .getOrCreate()

24/05/23 01:56:50 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
24/05/23 01:56:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 01:56:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = pd.read_csv('TopGlobalArtists50.csv')
selected_column = df['ArtistName']
selected_column.to_csv('ArtistList.csv', index=False)

In [33]:
artist_names_df = pd.read_csv("ArtistList.csv")
artist_data_list = []

for index, row in artist_names_df.iterrows():
    artist_name = row["ArtistName"]
    
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key=130ad717055f46130e0be133f8497c20&format=json"
    response = requests.get(url)
    artist_info = response.json()
    artist_data = {}
    if "artist" in artist_info:
        artist = artist_info["artist"]
        artist_data["ArtistName"] = artist.get("name")
        stats = artist.get("stats", {})
        artist_data["Listeners"] = stats.get("listeners")
        artist_data["PlayCount"] = stats.get("playcount")
        tags = artist.get("tags", {}).get("tag", [])
        genres = [tag["name"] for tag in tags]
        artist_data["Genres"] = ", ".join(genres)
        artist_data_list.append(artist_data)
    else:
        print(f"No artist information found for {artist_name}.")
artist_df = pd.DataFrame(artist_data_list)

display(artist_df)


,ArtistName,Listeners,PlayCount,Genres
0,Tommy Richman,605079,5827431,"rnb, Hip-Hop, rap, post-punk, funk"
1,Sabrina Carpenter,1567085,124375753,"pop, indie pop, alternative, folk, pop rock"
2,Kendrick Lamar,3649681,534405353,"Hip-Hop, rap, seen live, west coast, compton"
3,Billie Eilish,2843988,380244127,"pop, indie pop, seen live, electropop, indie"
4,Billie Eilish,2843988,380244127,"pop, indie pop, seen live, electropop, indie"
...,...,...,...,...
58,Taylor Swift,4650581,2346109905,"country, pop, female vocalists, singer-songwri..."
59,Lord Huron,1559832,40152300,"folk, seen live, indie, psychedelic, indie pop"
60,Luis R Conriquez,70224,2243133,"mexico, latin, español, corridos, REGIONAL MEX..."
61,Neton Vega,372,5004,
